In [1]:
from googletrans import Translator
import numpy as np
import pandas as pd
import time
import gezi
from gezi import tqdm
import sys, os
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from src import util
pd.set_option('display.max_colwidth', 512)

No module named 'tensorflow.python.keras.layers.preprocessing'


In [2]:
import random
ep = 4
random.seed(1024 + ep)

In [3]:
root = '../input/feedback-prize-2021'

In [4]:
df = pd.read_feather(f'{root}/train_en.fea')

In [5]:
# df.head(5)

In [6]:
langs = ['en', 'fr', 'de', 'es']
probs = [0.5, 0.1, 0.2, 0.2]
prob_indexes = []
for i, prob in enumerate(probs):
  prob_indexes.extend([i] * int(prob * 100))
# prob_indexes

In [7]:
dlangs = []
for lang in langs[1:]:
  folds = 10
  dlang = pd.concat(
    [pd.read_csv(f'{root}/trans_{lang}_{fold}.csv') for fold in range(folds) if os.path.exists(f'{root}/trans_{lang}_{fold}.csv')]
  )
  dlang[lang] = dlang['target'].replace('[Sep]', '[SEP]').replace('[SEP]', '[BR]')
  dlang = dlang.sort_values(['pid'])
  dlangs.append(dlang)

In [8]:
for lang, dlang in zip(langs[1:], dlangs):
  df = pd.merge(df, dlang[['pid', lang]], on='pid', how='left')
  df = df.fillna('[NONE]')
  ic(lang, len(df[df[lang] != '[NONE]']) / len(df))
df.head(5)

ic| 2342678226.py:4 in <module>
    lang: 'fr'
    len(df[df[lang] != '[NONE]']) / len(df): 0.9316681890842659
ic| 2342678226.py:4 in <module>
    lang: 'de'
    len(df[df[lang] != '[NONE]']) / len(df): 0.9356074446220598
ic| 2342678226.py:4 in <module>
    lang: 'es'
    len(df[df[lang] != '[NONE]']) / len(df): 0.9446791504909797


,index,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,cluster,worker,fr,de,es
0,0,423A1CA112E2,13,0,0,0,1,0,3.0,Phones [BR] [BR],"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",0,2,8,67,[NONE],Telephone [SEP] [SEP],Telephones [Sep] [Sep]
1,1,423A1CA112E2,13,1,5,1,45,3,47.0,Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving.,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",1,2,8,67,Modern humans today are always on their phone. They are always on their phone more than 5 hours a day without stopping. They do it even while driving.,Modern people are always on their phone today. They are always on their phone more than 5 hours a day not up to date.Everything you do is text back and guide themselves and just have group chats on social media. They even do it while driving.,"Modern humans today are always on your phone. They are always on your phone over 5 hours a day, do not stop.All they do is the text backward and forward and they only have group chats in social networks. Even they do it while driving."
2,2,423A1CA112E2,13,2,3,45,60,47,62.0,They are some really bad consequences when stuff happens when it comes to a phone.,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",2,2,8,67,These are really bad consequences when things happen when it comes to a phone.,They are some really bad consequences when things happen to the phone when it comes to a phone.,They are some really bad consequences when things happen when it comes to a phone.
3,3,423A1CA112E2,13,3,2,60,76,62,80.0,Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR],"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",3,2,8,67,Some areas of the United States prohibit classroom phones just because of that. [SEP] [SEP],Some specific areas in the United States banishes phones from classrooms only because of this. [Sep] [SEP],Some areas determined in the United States prohibit the telephones of classrooms just for it. [Sep] [Se

In [9]:
paras = []
choices = []
id = None
for row in tqdm(df.itertuples(), total=len(df)):
  row = row._asdict()
  candidates = [row['para']] + [row[lang] for lang in langs[1:]]  
  for i in range(len(candidates)):
    if candidates[i] == '[NONE]':
      candidates[i] = candidates[0]
  if len(str(row['para'].replace('[BR]', '')).split(' ')) <= 10:
    idx = 0
  else:
    idx = random.choice(prob_indexes)
  paras.append(candidates[idx])
  choices.append(langs[idx])
df['para'] = paras
df['choice'] = choices
df.head(5)

  0%|          | 0/175160 [00:00<?, ?it/s]

,index,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,cluster,worker,fr,de,es,choice
0,0,423A1CA112E2,13,0,0,0,1,0,3.0,Phones [BR] [BR],"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",0,2,8,67,[NONE],Telephone [SEP] [SEP],Telephones [Sep] [Sep],en
1,1,423A1CA112E2,13,1,5,1,45,3,47.0,Modern humans today are always on their phone. They are always on their phone more than 5 hours a day without stopping. They do it even while driving.,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",1,2,8,67,Modern humans today are always on their phone. They are always on their phone more than 5 hours a day without stopping. They do it even while driving.,Modern people are always on their phone today. They are always on their phone more than 5 hours a day not up to date.Everything you do is text back and guide themselves and just have group chats on social media. They even do it while driving.,"Modern humans today are always on your phone. They are always on your phone over 5 hours a day, do not stop.All they do is the text backward and forward and they only have group chats in social networks. Even they do it while driving.",fr
2,2,423A1CA112E2,13,2,3,45,60,47,62.0,They are some really bad consequences when things happen to the phone when it comes to a phone.,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",2,2,8,67,These are really bad consequences when things happen when it comes to a phone.,They are some really bad consequences when things happen to the phone when it comes to a phone.,They are some really bad consequences when things happen when it comes to a phone.,de
3,3,423A1CA112E2,13,3,2,60,76,62,80.0,Some areas of the United States prohibit classroom phones just because of that. [SEP] [SEP],"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",3,2,8,67,Some areas of the United States prohibit classroom phones just because of that. [SEP] [SEP],Some specific areas in the United States banishes phones from classrooms only because of this. [Sep] [SEP],Some areas determined in the United States prohibit the telephones of classrooms just for it. [Sep] [Sep],fr
4,4,423A1CA112E2,13,4,2,76,139,80,145.0,"W

In [10]:
ids = []
texts = []
id = None
words = []
for row in tqdm(df.itertuples(), total=len(df)):
  row = row._asdict()
  if row['id'] != id:
    if words:
      ids.append(id)
      texts.append(' '.join(words))
    words = []
    id = row['id']
  col = 'para'
  words.extend(str(row[col]).split(' '))
if words:
  ids.append(id)
  texts.append(' '.join(words))
d_text = pd.DataFrame({
  'id': ids,
  'text': texts
})
d_text

  0%|          | 0/175160 [00:00<?, ?it/s]

,id,text
0,423A1CA112E2,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day without stopping. They do it even while driving. They are some really bad consequences when things happen to the phone when it comes to a phone. Some areas of the United States prohibit classroom phones just because of that. [SEP] [SEP] When people have phones, they know about certain applications that have .Apps as Facebook Twitter Instagram and Snapchat. So, as if a friend moves away ..."
1,A8445CABFECE,"Phones & Driving [BR] [BR] Drivers should not be able to use phones while operating a vehicle. Drivers who used their phone while operating a vehicle are most likely to get into an accident that could be fatal. [BR] [BR] According to an article by the Edgar Snyder Firm, 21% of teens that were part of a fatal car accident was due to phones. According to the same article, 35% know the risk but continue using their phones while on the road. This shows that its beyond dangerous and irresponsible of drivers ..."
2,6B4F7A0165B9,"Cell Phone Operation While Driving [BR] [BR] The ability to stay connected to people we know despite distance was originally brought to fruition by the use of letters. This system was found to be rather slow and new pathways were searched for until the invention of the telegram; the people thought it was an invention of the millennia, but after time it too was thought of as slow until the invention of the telephone. Today, a telephone is in the hand or pocket of a majority of the seven billion people on..."
3,E05C7F5C1156,"People discuss whether the drivers do not use or should not use mobile phones during operation of a vehicle or should do not believe that the use of a mobile phone should be allowed during operation of a vehicle as they try to try multi-tasking and have important callsWhile some are addicted and can not stay the phone. Other people say they should not be on their phone during operation of a vehicle, as the mobile phone can cause accidents while driving. I also think that you shouldn't use your phone whi..."
4,50B3435E475B,"Texting and driving [BR] [BR] Over half of the riders in today's society has to fix this terrible habit of fixing SMS and driving. This must stop immediately, SMS and driving brings people's life. This simple file has caused many tatalities in recent years. Has happilyThe government set a new law to deal with this problem with our generation problem of the SMS problem and driving. I think firmly that texts and driving are dangerous and should be prohibited. [SEP] [SEP] Diving more into detail, texting a..."
...,...,...
15589,8F4B595CF9E7,"Do you ever want more opinions and options when it comes to tips? Well you can! You can simply ask for more than one person for your opinion so you can choose yours! It is good to go to multiple sources for opinions, because it opens your mind up to new things, you can get good or bad feed back, and Now you can see things from different points of view. [Sep] [Sep] The search for more opinions can open your opinion for new and other things. This can help you think outside the box and form new thoughts on..."
15590,6B5809C83978,"Has anyone ever given to them? Was the Council for the situation helpful? Seeking multiple opinions can help someone make a better choice. Getting different opinions can help someone make a better decision because they will get multiple answers, different types of advice, and You will have more information about the situation you have started. [Sep] [Sep] By looking for several opinions, people will get different answers and possibly make a better choice. Example, I struggled to decide what I should do ..."
15591,408A7D3D2EEC,"Imagine seeking advice from multiple people and choosing the best decision that suits you. How does that make you feel? Asking for advice can be defined as someone who asks help from multiple people and makes the right decision. Seeking multiple op

In [11]:
df = pd.merge(df, d_text, on=['id'], how='inner', suffixes=('_ori', ''))

In [12]:
starts, ends = [], []
end = 0
for row in tqdm(df.itertuples(), total=len(df)):
  row = row._asdict()
  if row['para_id'] == 0:
    end = 0
  start = end
  col = 'para' 
  end = start + len(str(row[col]).replace('[SEP]', '\n').split())
  starts.append(start)
  ends.append(end)
df['start'] = starts
df['end'] = ends
df.head(100)

  0%|          | 0/175160 [00:00<?, ?it/s]

,index,id,part,para_id,para_type,start,end,start2,end2,para,text_ori,pid,kfold,cluster,worker,fr,de,es,choice,text
0,0,423A1CA112E2,13,0,0,0,3,0,3.0,Phones [BR] [BR],"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",0,2,8,67,[NONE],Telephone [SEP] [SEP],Telephones [Sep] [Sep],en,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day without stopping. They do it even while driving. They are some really bad consequences when things happen to the phone when it comes to a phone. Some areas of the United States prohibit classroom phones just because of that. [SEP] [SEP] When people have phones, they know about certain applications that have .Apps as Facebook Twitter Instagram and Snapchat. So, as if a friend moves away ..."
1,1,423A1CA112E2,13,1,5,3,31,3,47.0,Modern humans today are always on their phone. They are always on their phone more than 5 hours a day without stopping. They do it even while driving.,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",1,2,8,67,Modern humans today are always on their phone. They are always on their phone more than 5 hours a day without stopping. They do it even while driving.,Modern people are always on their phone today. They are always on their phone more than 5 hours a day not up to date.Everything you do is text back and guide themselves and just have group chats on social media. They even do it while driving.,"Modern humans today are always on your phone. They are always on your phone over 5 hours a day, do not stop.All they do is the text backward and forward and they only have group chats in social networks. Even they do it while driving.",fr,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day without stopping. They do it even while driving. They are some really bad consequences when things happen to the phone when it comes to a phone. Some areas of the United States prohibit classroom phones just because of that. [SEP] [SEP] When people have phones, they know about certain applications that have .Apps as Facebook Twitter Instagram and Snapchat. So, as if a friend moves away ..."
2,2,423A1CA112E2,13,2,3,31,49,47,62.0,They are some really bad consequences when things happen to the phone when it comes to a phone.,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",2,2,8,67,These are really bad consequences when things happen when it comes to a phone.,They are some really bad consequences when things happen to the phone when it comes to a phone.,They are some really bad consequences when things ha

In [13]:
df[['id', 'para_type', 'start', 'end', 'text']].reset_index().to_feather(f'{root}/train_lang_ep{ep}.fea')